In [0]:
dbutils.widgets.dropdown("reset_all_data", "false", ["true", "false"], "Reset all data")

In [0]:
import requests
import collections
import os

class Utils():
    @staticmethod
    def setup_schema(catalog, db, reset_all_data, volume_name = None):
        if reset_all_data:
            print(f'clearing up volume named `{catalog}`.`{db}`.`{volume_name}`')
            try:
                spark.sql(f"DROP VOLUME IF EXISTS `{catalog}`.`{db}`.`{volume_name}`")
                spark.sql(f"DROP SCHEMA IF EXISTS `{catalog}`.`{db}` CASCADE")
            except Exception as e:
                print(f'catalog `{catalog}` or schema `{db}` do not exist.  Skipping data reset')

        def use_and_create_db(catalog, dbName, cloud_storage_path = None):
            print(f"USE CATALOG `{catalog}`")
            spark.sql(f"USE CATALOG `{catalog}`")
            spark.sql(f"""create database if not exists `{dbName}` """)

        assert catalog not in ['hive_metastore', 'spark_catalog'], "This demo only support Unity. Please change your catalog name."
        #If the catalog is defined, we force it to the given value and throw exception if not.
        current_catalog = spark.sql("select current_catalog()").collect()[0]['current_catalog()']
        if current_catalog != catalog:
            catalogs = [r['catalog'] for r in spark.sql("SHOW CATALOGS").collect()]
            if catalog not in catalogs:
                spark.sql(f"CREATE CATALOG IF NOT EXISTS `{catalog}` MANAGED LOCATION 'abfss://unity-catalog-storage@dbstoragefuavvebhyzyxm.dfs.core.windows.net/3759185753378633'")
                if catalog == 'dbdemos':
                    spark.sql(f"ALTER CATALOG `{catalog}` OWNER TO `account users`")
        
        use_and_create_db(catalog, db)

        if catalog == 'dbdemos':
            try:
                spark.sql(f"GRANT CREATE, USAGE on DATABASE `{catalog}`.`{db}` TO `account users`")
                spark.sql(f"ALTER SCHEMA `{catalog}`.`{db}` OWNER TO `account users`")
                for t in spark.sql(f'SHOW TABLES in {catalog}.{db}').collect():
                    try:
                        spark.sql(f'GRANT ALL PRIVILEGES ON TABLE {catalog}.{db}.{t["tableName"]} TO `account users`')
                        spark.sql(f'ALTER TABLE {catalog}.{db}.{t["tableName"]} OWNER TO `account users`')
                    except Exception as e:
                        if "NOT_IMPLEMENTED.TRANSFER_MATERIALIZED_VIEW_OWNERSHIP" not in str(e) and "STREAMING_TABLE_OPERATION_NOT_ALLOWED.UNSUPPORTED_OPERATION" not in str(e) :
                            print(f'WARN: Couldn t set table {catalog}.{db}.{t["tableName"]} owner to account users, error: {e}')
            except Exception as e:
                print("Couldn't grant access to the schema to all users:"+str(e))    

        print(f"using catalog.database `{catalog}`.`{db}`")
        spark.sql(f"""USE `{catalog}`.`{db}`""")    

        if volume_name:
            spark.sql(f'CREATE VOLUME IF NOT EXISTS {volume_name};')

    @staticmethod
    def wait_for_table(table_name, timeout_duration=120):
        import time
        i = 0
        while not spark.catalog.tableExists(table_name) or spark.table(table_name).count() == 0:
            time.sleep(1)
            if i > timeout_duration:
                raise Exception(f"couldn't find table {table_name} or table is empty. Do you have data being generated to be consumed?")
            i += 1

    @staticmethod
    def get_active_streams(start_with = ""):
        return [s for s in spark.streams.active if len(start_with) == 0 or (s.name is not None and s.name.startswith(start_with))]

    @staticmethod
    def stop_all_streams(start_with = "", sleep_time=0):
        import time

        time.sleep(sleep_time)
        streams = Utils.get_active_streams(start_with)

        if len(streams) > 0:
            print(f"Stopping {len(streams)} streams")

            for s in streams:
                try:
                    s.stop()
                except:
                    pass
                
            print(f"All stream stopped {'' if len(start_with) == 0 else f'(starting with: {start_with}.)'}")
